In [19]:
import sys
print(sys.version)

3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]


In [20]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Uninstalling openai-1.6.1:
      Successfully uninstalled openai-1.6.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


### Provide your OpenAI API key. Hardcode it if not saved in the system enviroment, then 

In [124]:
from openai import OpenAI
import os

api_key = os.environ["openai_api_key"]     #  paid api
#api_key = "paste your openai_api_key here" 

In [59]:
client = OpenAI(api_key=api_key) 

### Loading instructions document

#### These are the instructions for the LLM to follow (such as what to display, use only the presented infomation to make decision, what to do incase of missing information, chain of thoughts to show how it arived at the results and end with a conclusion in less than 40 words)

In [26]:
instruction_file = client.files.create(
    file=open("instructions/instructions.docx", 'rb'),
    purpose="assistants"

)

In [27]:
print(instruction_file)

FileObject(id='file-rju80R44NNvsu85WQ7BHnXqF', bytes=15539, created_at=1704794568, filename='instructions.docx', object='file', purpose='assistants', status='processed', status_details=None)


### Loading Guidelines document

In [86]:
guidelines_file  =  client.files.create(
    file=open("instructions/colonoscopy-guidelines.pdf", 'rb'),
    purpose="assistants"

)

In [87]:
print(guidelines_file)

FileObject(id='file-uvLfXb29jJFNXttz7JGoLzYD', bytes=42393, created_at=1704811629, filename='colonoscopy-guidelines.pdf', object='file', purpose='assistants', status='processed', status_details=None)


### Loading Patient's medical record document

#### Load the appropriate medical record through the popup

In [126]:
import tkinter as tk
from tkinter import filedialog

# Function to open file dialog and get file path
def upload_file():
    #print("Welcome to the File Uploader")
    print("Please select the Patient Medical Record for pre-authorization check: ")

    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename()
    if file_path:  # Check if a file was selected
        print("Selected file path:", file_path)    
    else:
        print("No file was selected.")
    root.destroy()
    
    return file_path


file_path =upload_file()
#file_path

Please select the Patient Medical Record for pre-authorization check: 
Selected file path: /Users/abhi/Documents/medical-record-3.pdf


In [107]:
patient_file=client.files.create(
    file=open(file_path, 'rb'),    #file=open("medical-record-1.pdf", 'rb'),
    purpose="assistants"

)

In [108]:
print(patient_file)

FileObject(id='file-OyStnTsdv0bsB5oOWESUbFVd', bytes=67330, created_at=1704899282, filename='medical-record-3.pdf', object='file', purpose='assistants', status='processed', status_details=None)


### Making OpenAI API call. Using 'gpt-4-1106-preview' model and retrieval tool

In [109]:
assistant = client.beta.assistants.create(
    name = "Medical_PreAuthBOT",
    instructions = "You are an expert medical assistant. Follow the instructions listed here to answer if the requested procedure(s) for the patient should be approved. If not provide details of the additional information that may help in making decision.1.	Please use only the information provided to answer. 2.	Your goal is to approve the requested procedure only when all the required criterias are met.3.	Ingest patient medical record PDF (medical-record-x.pdf) where X is likely a number. 4.	Create a timeline of patient’s medical history on complaints, diagnostics, diagnosis, procedures, treatments, medications etc.5.	Extract all the CPT code(s) from this document which is usually present after the text ‘Requested Procedure’ to identify which procedure(s) have been recommended by the doctor.6.	Display the name of the patient, patient’s date of birth or DOB, calculate the age from the date of birth or DOB and display it. Display MRN if provided. 7.	Identify if any conservative treatment has already been attempted from the patient medical record PDFs whether in medical procedures, clinical procedures or notes.8.	If a prior conservative treatment has already been attempted and if the treatment was successful or have shown signs of improvements, then present evidence that conservative treatment improved the patient’s condition and disapprove the need for the Requested Procedure stating the reason.9.	If the conservative treatment was not found or has failed then mention explicitly that ‘conservative treatment was not found or has failed’, and then look for the criterias present for Colorectal cancer screening, as indicated by 1 or more of the following to identify if the Requested Procedure should be allowed or not. Please state which condition(s) in the guidelines were used to arrive at the answer.a.	Patient has average-risk or higher, if Age is 45 years or older and No colonoscopy in past 10 yearsb.	Patient has High risk family history, as indicated by 1 or more of the following:1.	Colorectal cancer diagnosed in one or more first-degree relatives of any age and Age 40 years or older and Symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding)2.	Family member with colonic adenomatous polyposis of unknown etiologyc.	Juvenile polyposis syndrome diagnosis indicated by 1 or more of the following:1.	Age 12 years or older and symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia)2.	Age younger than 12 years and symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia)10.	At any point if you don’t know the answer, just say ‘I cannot arrive at the conclusion. Please provide additional information’. Do not hallucinate or provide incorrect or incomplete information. Please request specific information that could be helpful in making a decision.11.	Walk me through the process with chain-of-thoughts on how you arrived at the conclusion. 12.	At the end, after two blank line, summarise the final conclusion with the title ‘Conclusion:’ in less than 40 words if the requested procedure should be approved or not. Provide the reason why it should be approved or not approved. In case of indecision, specify what further information would be required to decide.",
    tools = [{"type":"retrieval"}],
    model = "gpt-4-1106-preview",
    file_ids = [patient_file.id] # In this version I have copy pasted the guidelines into the instructions 
    # I used retireval by uploading guidelines in an earlier version. 
    # I have provided the results as screenshot in the word document emailed
)

#assistant

In [111]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_fkYDVLzSLNTb4mBplKomPGvT', created_at=1704899284, metadata={}, object='thread')


In [112]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "You are a helpful expert medical assistant. Use the information provided to answer."

)

In [113]:
message

ThreadMessage(id='msg_aQ6xuD4TRQTU8CAF7EqocnO5', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='You are a helpful expert medical assistant. Use the information provided to answer.'), type='text')], created_at=1704899284, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_fkYDVLzSLNTb4mBplKomPGvT')

In [114]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= assistant.id

)
#run

### Waiting for run to complete and checking the status

In [116]:
import time 

while True:
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    time.sleep(10)
    
    # Check the status of the run
    if run_status.status == 'completed':
        # If completed, break the loop and process messages
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        break
    elif run_status.status in ['failed', 'error']:
        # If the run fails or encounters an error, handle it appropriately
        print(f"Run encountered an error: {run_status.status}")
        break
    else:
        # If still running, continue to wait
        time.sleep(2)


### If the run completed successfully, prinitng the results, else printing the error message 

In [117]:
if run_status.status in ['failed', 'error']:
    print(f"Run encountered an error: {run_status.status}")
else: 
    for message in reversed(messages.data):
      print(message.role + ":" + message.content[0].text.value)

user:You are a helpful expert medical assistant. Use the information provided to answer.
assistant:Patient Name: Michail Antonio
DOB: 01/01/1975
MRN: Not provided

Age: 48 years old (as of 12/06/2023)

Medical History Timeline:
- Hypertension diagnosed in 2015, controlled with medication (Lisinopril 10mg daily).
- Cholecystectomy performed on 08/15/2018 (CPT 47562).
- Colonoscopy performed on 07/22/2020 (CPT 45378), with abnormal findings and pending further treatment.
- Since May 2023, the patient has been experiencing persistent upper abdominal pain and heartburn, which worsen post-meal and especially after consuming spicy foods (over-the-counter antacids provide minimal relief).

Requested Procedure(s):
- CPT 43235
- CPT 43239

Conservative treatments:
- The patient has been taking antacids as needed since June 2023 which have provided minimal relief.

Regarding the requested procedures, the diagnostic colonoscopy (CPT 45378) was performed in July 2020, and the history indicated abn

## Code ends here. I have saved the output below to demo the results for the 3 Patients

### If you run the cells below, it will be over-written with the current patient medical record

# Patient 1

In [71]:
if run_status.status in ['failed', 'error']:
    print(f"Run encountered an error: {run_status.status}")
else: 
    for message in reversed(messages.data):
      print(message.role + ":" + message.content[0].text.value)

user:You are a helpful expert medical assistant. Use the information provided to answer.
assistant:The patient's name is James Freeman 【7†source】. His date of birth is 06/16/1982, which makes him 40 years old as of the current year, 2023 【8†source】. The Medical Record Number (MRN) is 456789123 【7†source】.

Creating a timeline of patient’s medical history, we have the following key points:

- Complaint: James has occasional rectal bleeding and abdominal discomfort for the past 6 months 【9†source】.
  
- Family History: His father had colorectal cancer at age 68. 

- Personal Medical History: Hypertension managed with medication.

- Medications: He is on Lisinopril 10mg daily.

- Past Treatments: James had an appendectomy in 2015 and a knee arthroscopy in 2018 (CPT codes 44970 and 29881 respectively).

- Notes indicate that the patient underwent treatment for abdominal discomfort one year ago without significant improvement. A repeat treatment 6 weeks ago led to noticeable improvement in 

# Patient 2

In [92]:
if run_status.status in ['failed', 'error']:
    print(f"Run encountered an error: {run_status.status}")
else: 
    for message in reversed(messages.data):
      print(message.role + ":" + message.content[0].text.value)

user:You are a helpful expert medical assistant. Use the information provided to answer.
assistant:Patient Medical History Timeline:
- Family History: Father had colorectal cancer at a significantly advanced age.
- Personal Medical History: Hypertension managed with medication.
- Medications: Lisinopril 10 mg daily and possibly other medications not listed.
- Allergies: No known drug allergies.
- Presenting Complaint: Occasional rectal bleeding and abdominal discomfort for a few months.
- Physical Examination: Tenderness in the lower abdomen.
- Diagnostic Tests: Hemoglobin levels low at 12.4 g/dL.
- Previous Medical Procedures: Gallbladder removal (CPT Code 47562), physical therapy for lower back pain and abdominal symptoms.
- Clinical Impression: Internal hemorrhoids suggested by the patient.
- Planned Procedure: Colonoscopy (CPT Code 45378) on 12/15/2023.

CPT code extracted from the document is 45378, which relates to the requested procedure of a colonoscopy.

Name of the patient: J

# Patient 3

In [118]:
if run_status.status in ['failed', 'error']:
    print(f"Run encountered an error: {run_status.status}")
else: 
    for message in reversed(messages.data):
      print(message.role + ":" + message.content[0].text.value)

user:You are a helpful expert medical assistant. Use the information provided to answer.
assistant:Patient Name: Michail Antonio
DOB: 01/01/1975
MRN: Not provided

Age: 48 years old (as of 12/06/2023)

Medical History Timeline:
- Hypertension diagnosed in 2015, controlled with medication (Lisinopril 10mg daily).
- Cholecystectomy performed on 08/15/2018 (CPT 47562).
- Colonoscopy performed on 07/22/2020 (CPT 45378), with abnormal findings and pending further treatment.
- Since May 2023, the patient has been experiencing persistent upper abdominal pain and heartburn, which worsen post-meal and especially after consuming spicy foods (over-the-counter antacids provide minimal relief).

Requested Procedure(s):
- CPT 43235
- CPT 43239

Conservative treatments:
- The patient has been taking antacids as needed since June 2023 which have provided minimal relief.

Regarding the requested procedures, the diagnostic colonoscopy (CPT 45378) was performed in July 2020, and the history indicated abn